In [1]:
import sqlalchemy
import math
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta


In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=b013c5c0322596ecf6b14c7fb161260df46b8c6d584988bea0a1711e731dd957
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [32]:
db = sql.read.csv("/content/drive/MyDrive/DBMS_project_final/Source Code/1-Datasets/KCPD_Crime_Data/zip_code_database.csv",)

In [3]:
def build_metadata():
    metadata = sqlalchemy.MetaData(db)
    metadata.bind.echo = False
    metadata.bind.text_factory = str

    return metadata

In [4]:
def select_zipcode(zipcode):
    fields = (
        'zip',
        'city',
        'state',
        'lat',
        'long',
        'timezone',
        'dst',
        )
    metadata = build_metadata()
    zipcodes_table = sqlalchemy.Table('zipcodes', metadata, autoload=True)
    result = zipcodes_table.select(zipcodes_table.c.zip == zipcode)
    try:
        return dict(zip(fields,result.execute().fetchone()))
    except (TypeError, sqlalchemy.exc.OperationalError):
        return False

In [5]:
def distance(zipcode1, zipcode2):
    z1 = select_zipcode(zipcode1)
    z2 = select_zipcode(zipcode2)
    if not(z1) or not(z2):
        return (99999999)
    return haversine(z1['lat'], z1['long'], z2['lat'], z2['long'])


In [6]:
def long_lat(zipcode1):
    z1 = select_zipcode(zipcode1)
    if not(z1):
        return ["NA", "NA"]
    return [z1['lat'], z1['long']]

In [7]:
def haversine(lat1, long1, lat2, long2):
    radius = 3963.1676 #Radius of earth in miles
    lat1, long1, lat2, long2 = map(math.radians, [lat1, long1, lat2, long2])
    dlat = lat2 - lat1
    dlong = long2 - long1

    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlong/2) * math.sin(dlong/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [8]:
def span(today, prev):
    m1, d1, y1 = [int(x) for x in (prev).split('/')]
    m2, d2, y2 = today.month,today.day,today.year
    prev1 = date(y1, m1, d1)
    today1 = date(y2, m2, d2)

    if today1<prev1:
        return True
    return False

In [16]:
import os
import ast
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.10/dist-packages/pyspark"
os.environ["HADOOP_HOME"]="C:/winutils"
from operator import add
import pdb
vertex=1

In [10]:
from pyspark import SparkContext

from pyspark.sql import functions as sf
from pyspark.sql.functions import udf

In [19]:
from pyspark.sql import SQLContext
# sc = SparkContext('local')
sql = SQLContext(sc)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [21]:
df = sql.read.csv("/content/drive/MyDrive/DBMS_project_final/Source Code/1-Datasets/KCPD_Crime_Data/KCcrime2010To2018.csv", inferSchema = True, header = True)

In [22]:
df = df.withColumn('Reported_date', sf.concat(sf.col('Reported_month'),sf.lit('/'),sf.col('Reported_day'),sf.lit('/'),sf.col('Reported_year')))

In [23]:
data=df.toPandas()

In [24]:
data.head()

,Report_No,Reported_year,Reported_month,Reported_day,Reported_hour,Reported_minute,From_year,From_month,From_day,From_hour,...,Rep_Dist,Area,DVFlag,Invl_No,Involvement,Race,Sex,Age,Firearm_Used_Flag,Reported_date
0,100013768,2010,2,24,12,12,2010,2,24,12,...,PJ7172,SPD,U,2,VIC,U,U,36.0,N,2/24/2010
1,100002393,2010,1,11,2,0,2010,1,10,23,...,PJ7294,SPD,Y,1,VIC,W,F,16.0,N,1/11/2010
2,100041100,2010,6,2,17,48,2010,6,2,17,...,PJ4317,MPD,N,1,VIC,B,F,16.0,N,6/2/2010
3,100000548,2010,1,3,14,1,2009,12,29,19,...,PJ2966,EPD,U,1,SUS,U,U,36.0,N,1/3/2010
4,100013505,2010,2,23,11,34,2010,2,16,20,...,PP0375,NPD,U,2,VIC,W,M,34.0,N,2/23/2010


In [26]:
data.columns

Index(['Report_No', 'Reported_year', 'Reported_month', 'Reported_day',
       'Reported_hour', 'Reported_minute', 'From_year', 'From_month',
       'From_day', 'From_hour', 'From_minute', 'Offense', 'IBRS',
       'Description', 'Beat', 'Address', 'City', 'Zip_Code', 'Rep_Dist',
       'Area', 'DVFlag', 'Invl_No', 'Involvement', 'Race', 'Sex', 'Age',
       'Firearm_Used_Flag', 'Reported_date'],
      dtype='object')

In [27]:
data['Reported_date']

0           2/24/2010
1           1/11/2010
2            6/2/2010
3            1/3/2010
4           2/23/2010
              ...    
1121569    12/26/2018
1121570    12/23/2018
1121571    12/27/2018
1121572    12/29/2018
1121573    12/25/2018
Name: Reported_date, Length: 1121574, dtype: object

In [35]:
def temporary(data):
    days = 0
    person=[1750, 802, 302, 301, 402, 401, 2001, 201, 1198, 2655, 799, 101, 610, 1701, 2661, 2662, 1770, 3071, 3067, 2530, 3074, 3009]
    months = 6
    crime_person=0
    crime_prop=0
    years = 1
    isSafe=0
    Safety=0
    time=input("How many months worth of data do you want to see?: ")
    months=int(time)
    for te, de in zip(data["Offense"].unique(), data["Description"].unique()):
        print(te," ", de)
    type=input("WHICH TYPE OF CRIME? enter number or x for all types: ")
    if type!='x':
        type=int(type)
    since = date.today() - relativedelta(days=days, months=months, years=years)
    threshold = int(input("Please enter the Radius: "))
    locality = input("Please enter the Location Zipcode: ")
    records=0
    for x, y in zip(data["Zip_Code"], data['Reported_date']):
        try:
            duration = span(since, y)
            dist = distance(int(locality), int(x))
            if (dist < threshold and duration):
                  print("Distance in between " + str(locality) + " %.2f at date: " % dist, y)
        except IndexError:
            print("arguements are 2 US Zip Codes\nzipcode_distance zipcode1 zipcode2")
    print("Total of {} records",records)
    print("SAFETY LEVE HERE is: ", 100-((crime_person/records)*70)-((crime_prop/records)*30))


if __name__ == "__main__":
    sc = SparkContext.getOrCreate()
    import sys
    import datetime

    duration = udf(temporary(data))

How many months worth of data do you want to see?: 3
1352   Stolen Property OFFE
802   Non Agg Assault Dome
801   Non Aggravated Assau
702   Auto Theft
1198   Impersonation
401   Aggravated Assault (
2100   Driving Under Influe
690   Stealing All Other
501   Burglary - Residence
2655   Trespassing
1850   Possession of Drug E
1401   Property Damage
2040   Violation Full Order
640   Stealing From Auto
201   Rape
2601   Misc Violation
670   Stealing from Buildi
2402   Disorderly Conduct
650   Stealing Auto Parts/
302   Armed Robbery
799   Auto Theft Outside S
403   Agg Assault - Domest
502   Burglary - Non Resid
1002   Forged Checks
1122   Credit/Debit/ATM Car
1849   Possession/Sale/Dist
630   Stealing Shoplifting
1702   Statutory Rape
680   Stealing Coin Operat
831   Intimidation
1130   Fraud Confidence Gam
1120   Fraud
301   Strong Arm Robbery
402   Agg Assault - Drive-
1546   Weapons Law Violatio
840   Resisting an Officer
2001   Family Offense
1188   Wire Fraud
1001   Forgery
210   Fo

ArgumentError: ignored

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
